In [6]:
import torch
from torch import nn
import copy
import math
import random
from collections import OrderedDict, defaultdict

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from tqdm.auto import tqdm
from torchsummary import summary
import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import torchvision.models as models
import torchvision
from torch.utils.data import DataLoader

from torchvision.datasets import *
from torchvision.transforms import *

from ptq.layers import QAct, QConv2d, QIntLayerNorm, QIntSoftmax, QLinear, QConvTranspose2d , QConv2d_BNFold ,QConvTranspose2d_BNFold , QConv2d_output

no_cuda = False
use_gpu = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


class h_sigmoid(nn.Module):
    def __init__(self, inplace=True):
        super(h_sigmoid, self).__init__()
        self.relu = nn.ReLU6(inplace=inplace)

    def forward(self, x):
        return self.relu(x + 3) / 6


class h_swish(nn.Module):
    def __init__(self, inplace=True):
        super(h_swish, self).__init__()
        self.sigmoid = h_sigmoid(inplace=inplace)

    def forward(self, x):
        return x * self.sigmoid(x)


class SELayer(nn.Module):
    def __init__(self, channel, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(channel, _make_divisible(channel // reduction, 8)),
                nn.ReLU(inplace=True),
                nn.Linear(_make_divisible(channel // reduction, 8), channel),
                h_sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(

        QConv2d_BNFold(inp,
                 oup,
                 kernel_size=3,
                 stride= stride,
                 padding=1,
                 dilation=1,
                 groups=1,
                 bias=True,
                 quant=True,
                 calibrate=False,
                 last_calibrate=False),

        h_swish()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        QConv2d_BNFold(inp,
                 oup,
                 kernel_size=1,
                 stride= 1,
                 padding=0,
                 dilation=1,
                 groups=1,
                 bias=True,
                 quant=True,
                 calibrate=False,
                 last_calibrate=False),
        h_swish()
    )


class InvertedResidual(nn.Module):
    def __init__(self, inp, hidden_dim, oup, kernel_size, stride, use_se, use_hs):
        super(InvertedResidual, self).__init__()
        assert stride in [1, 2]

        self.identity = stride == 1 and inp == oup

        if inp == hidden_dim:
            self.conv = nn.Sequential(
                # dw
                QConv2d_BNFold(hidden_dim,
                        hidden_dim,
                        kernel_size=kernel_size,
                        stride= stride,
                        padding=(kernel_size - 1) // 2,
                        dilation=1,
                        groups=hidden_dim,
                        bias=True,
                        quant=True,
                        calibrate=False,
                        last_calibrate=False),
                h_swish() if use_hs else nn.ReLU(inplace=True),
                # Squeeze-and-Excite
                SELayer(hidden_dim) if use_se else nn.Identity(),
                # pw-linear
                QConv2d_BNFold(hidden_dim,
                        oup,
                        kernel_size=1,
                        stride= 1,
                        padding=0,
                        dilation=1,
                        groups=1,
                        bias=True,
                        quant=True,
                        calibrate=False,
                        last_calibrate=False),                
            )
        else:
            self.conv = nn.Sequential(
                # pw
                QConv2d_BNFold(inp,
                        hidden_dim,
                        kernel_size=1,
                        stride= 1,
                        padding=0,
                        dilation=1,
                        groups=1,
                        bias=True,
                        quant=True,
                        calibrate=False,
                        last_calibrate=False),    
                h_swish() if use_hs else nn.ReLU(inplace=True),
                # dw
                QConv2d_BNFold(hidden_dim,
                        hidden_dim,
                        kernel_size=kernel_size,
                        stride= stride,
                        padding=(kernel_size - 1) // 2,
                        dilation=1,
                        groups=hidden_dim,
                        bias=True,
                        quant=True,
                        calibrate=False,
                        last_calibrate=False),
                # Squeeze-and-Excite
                SELayer(hidden_dim) if use_se else nn.Identity(),
                h_swish() if use_hs else nn.ReLU(inplace=True),
                # pw-linear
                QConv2d_BNFold(hidden_dim,
                        oup,
                        kernel_size=1,
                        stride= 1,
                        padding=0,
                        dilation=1,
                        groups=1,
                        bias=True,
                        quant=True,
                        calibrate=False,
                        last_calibrate=False), 
            )

    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class  our_MobileNetV3(nn.Module):
    def __init__(self, cfgs, mode, num_classes=10, width_mult=1.):
        super(our_MobileNetV3, self).__init__()
        # setting of inverted residual blocks
        self.cfgs = cfgs
        assert mode in ['large', 'small','Ours']

        # building first layer
        input_channel = 1
        layers = [conv_3x3_bn(3, input_channel, 2)]
        # building inverted residual blocks
        block = InvertedResidual
        for k, t, c, use_se, use_hs, s in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            exp_size = _make_divisible(input_channel * t, 8)
            layers.append(block(input_channel, exp_size, output_channel, k, s, use_se, use_hs))
            input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        self.conv = conv_1x1_bn(input_channel, exp_size)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        output_channel = {'large': 1280, 'small': 1024,'Ours':10}
        output_channel = _make_divisible(output_channel[mode] * width_mult, 8) if width_mult > 1.0 else output_channel[mode]
        self.classifier = nn.Sequential(
            nn.Linear(exp_size, output_channel),
            h_swish(),
            nn.Dropout(0.2),
            nn.Linear(output_channel, num_classes),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def Ours_mobilenetv3(**kwargs):
    """
    Constructs a Ours-MobileNetV3 model
    """
    cfgs = [
        # kernal_size, t, c, SE, HS, s 
        [3,    1,  16, 1, 1, 2],
        [3,  4.5,  24, 1, 1, 2],
        [3, 3.67,  24, 1, 1, 1],
        [5,    4,  40, 1, 1, 2],
        [5,    6,  40, 1, 1, 1],
        [5,    6,  40, 1, 1, 1],
        [5,    3,  48, 1, 1, 1],
        [5,    3,  48, 1, 1, 1],



    ]
    return our_MobileNetV3(cfgs, mode='Ours', **kwargs)

In [7]:
test =  Ours_mobilenetv3()

In [8]:
# Create an instance of the DNN model
net =  Ours_mobilenetv3().cuda()  # Move the model to GPU if available
print(net)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
grad_para = count_parameters(net)
# print(f'Total params: {grad_para / 1e6}M')
summary(net,(1,28,28))

#Compute MACs
from thop import profile
input1 = torch.randn(1,1,28,28).cuda()
MACs, params = profile(net, inputs=(input1, ))
print('MACs = ' + str(MACs/1000**3) + 'G')
print('Params = ' + str(params/1000**2) + 'M')

our_MobileNetV3(
  (features): Sequential(
    (0): Sequential(
      (0): QConv2d_BNFold(
        3, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
        (quantizer): UniformQuantizer()
        (quantizer_act): UniformQuantizer()
      )
      (1): h_swish(
        (sigmoid): h_sigmoid(
          (relu): ReLU6(inplace=True)
        )
      )
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): QConv2d_BNFold(
          1, 8, kernel_size=(1, 1), stride=(1, 1)
          (quantizer): UniformQuantizer()
          (quantizer_act): UniformQuantizer()
        )
        (1): h_swish(
          (sigmoid): h_sigmoid(
            (relu): ReLU6(inplace=True)
          )
        )
        (2): QConv2d_BNFold(
          8, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=8
          (quantizer): UniformQuantizer()
          (quantizer_act): UniformQuantizer()
        )
        (3): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (fc):

AttributeError: 'NoneType' object has no attribute 'reshape'